In [1]:
import networkx as nx
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags

from scipy.stats import spearmanr

from util.models import MODELS
from util.tasks import TASKS

%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()
from attention_graph_util import *



chkpt_dir='../tf_ckpts'

task_name = 'word_sv_agreement_vp'
task_params = get_task_params()
task_params.batch_size = 1
task = TASKS[task_name](task_params, data_dir='../data')

Vocab len:  10034


In [2]:
# LSTM to Transformer VP
exp_name='samira_fd5'
teacher_exp_name='0.001_samira_offlineteacher_v5'
teacher_config='small_lstm_v4'
model_name='cl_bert'
teacher_model='cl_lstm'
model_config='small_gpt_v9'
distill_config='pure_distill_4_cosinerestart_slow'
distill_mode='offline'

# exp_name='samira_fd7'
# teacher_exp_name='0.001_samira_offlineteacher_v5'
# teacher_config='small_lstm_v4'
# model_name='cl_bert'
# teacher_model='cl_lstm'
# model_config='short_gpt_v9'
# distill_config='pure_distill_4_cosinerestart_slow'
# distill_mode='offline'


cl_token = task.databuilder.sentence_encoder().encode(constants.bos)
hparams=get_model_params(task, model_name, model_config)
hparams.output_attentions = True
hparams.output_embeddings = True
hparams.output_hidden_states = True
model_1 = MODELS[model_name](hparams=hparams, cl_token=cl_token)
teacher_model = MODELS[teacher_model](hparams=get_model_params(task, teacher_model, teacher_config), cl_token=cl_token)


ckpt_dir = os.path.join(chkpt_dir, task.name,
                              '_'.join([distill_mode,distill_config,
                                        "teacher", teacher_model.model_name, 
                                        teacher_config,
                                        teacher_exp_name,
                                       "student",model_1.model_name,
                                        str(model_config),
                                        exp_name]))

ckpt = tf.train.Checkpoint(net=model_1)
manager = tf.train.CheckpointManager(ckpt, ckpt_dir, max_to_keep=None)

ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  print("Restored student from {}".format(manager.latest_checkpoint))

model_1.compile(loss=task.get_loss_fn(), metrics=task.metrics())
x, y = iter(task.valid_dataset).next()
out = model_1(x)
print(out.shape)
#model_1.evaluate(task.test_dataset, steps=100)


model config: small_gpt_v9
{'embedding_dim': 128, 'resid_pdrop': 0.4, 'embd_pdrop': 0.2, 'attn_pdrop': 0.6, 'initializer_range': 0.05}
model config: small_lstm_v4
{'hidden_dim': 256, 'embedding_dim': 256, 'depth': 2, 'hidden_dropout_rate': 0.8, 'input_dropout_rate': 0.2, 'initializer_range': 0.1}
Restored student from ../tf_ckpts/word_sv_agreement_vp/offline_pure_distill_4_cosinerestart_slow_teacher_cl_lstm_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_0.001_samira_offlineteacher_v5_student_cl_bert_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_v9_samira_fd5/ckpt-60
(1, 2)


In [ ]:
# LSTM to Transformer VP
exp_name='samira_fd5'
teacher_exp_name='0.001_samira_offlineteacher_v5'
teacher_config='small_lstm_v4'
model_name='cl_bert'
teacher_model='cl_lstm'
model_config='small_gpt_v9'
distill_config='pure_distill_4_cosinerestart_slow'
distill_mode='offline'

cl_token = task.databuilder.sentence_encoder().encode(constants.bos)
hparams=get_model_params(task, model_name, model_config)
hparams.output_attentions = True
hparams.output_embeddings = True
hparams.output_hidden_states = True
model_2 = MODELS[model_name](hparams=hparams, cl_token=cl_token)
teacher_model = MODELS[teacher_model](hparams=get_model_params(task, teacher_model, teacher_config), cl_token=cl_token)


ckpt_dir = os.path.join(chkpt_dir, task.name,
                              '_'.join([distill_mode,distill_config,
                                        "teacher", teacher_model.model_name, 
                                        teacher_config,
                                        teacher_exp_name,
                                       "student",model_2.model_name,
                                        str(model_config),
                                        exp_name]))

ckpt = tf.train.Checkpoint(net=model_2)
manager = tf.train.CheckpointManager(ckpt, ckpt_dir, max_to_keep=None)

ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  print("Restored student from {}".format(manager.latest_checkpoint))

model_2.compile(loss=task.get_loss_fn(), metrics=task.metrics())
x, y = iter(task.valid_dataset).next()
out = model_2(x)
print(out.shape)


In [3]:
v_iter = iter(task.valid_dataset)

In [4]:
x, y = v_iter.next()

In [5]:

index = 0
sentence =  task.databuilder.sentence_encoder().decode(x[index])#'<bos> the young girl who plays with the older kids is happy <eos>'
#sentence = '<bos> the young girl who plays with the older kids <eos>'
print(sentence)
#'<bos> the boys who play football with the old man <eos>'
encoded_sentence = task.databuilder.sentence_encoder().encode(sentence)
logits, extra = model_1.detailed_call(np.asarray([encoded_sentence]), training=False)
print((np.argmax(logits) == [0]))
logits, last_state, presents, attentions, embeddings = extra
_attentions = [att.numpy() for att in attentions]
attentions_mat = np.asarray(_attentions)[:,0]
print(attentions_mat.shape)

<bos> be ready to serve at every opportunity , yet making sure that your fellow servers <eos>
[False]
(6, 8, 18, 18)


In [ ]:
att_mat = attentions_mat.sum(axis=1)
adj_mat, labels_to_index = get_adjmat(mat=att_mat, input_tokens=['<cls>']+sentence.split())

G = draw_attention_graph(adj_mat,labels_to_index, n_layers=attentions_mat.shape[0], length=attentions_mat.shape[-1])

In [ ]:
plt.savefig('raw_attentions.png', transparent=True)

In [ ]:
res_att_mat = attentions_mat.sum(axis=1)
res_att_mat = res_att_mat + 8*np.eye(res_att_mat.shape[1])[None,...]
res_att_mat = 8*res_att_mat / res_att_mat.sum(axis=-1)[...,None]
 
res_adj_mat, res_labels_to_index = get_adjmat(mat=res_att_mat, input_tokens=['<cls>']+sentence.split())

res_G = draw_attention_graph(res_adj_mat,res_labels_to_index, n_layers=res_att_mat.shape[0], length=res_att_mat.shape[-1])

In [ ]:
output_nodes = []
input_nodes = []
for key in labels_to_index:
    if 'L6' in key:
        output_nodes.append(key)
    if labels_to_index[key] < attentions_mat.shape[-1]:
        input_nodes.append(key)

flow_values = compute_flows(res_G, res_labels_to_index, input_nodes, length=attentions_mat.shape[-1])
flow_G = draw_attention_graph(flow_values,labels_to_index, n_layers=attentions_mat.shape[0], length=attentions_mat.shape[-1])
#plt.savefig('flow_attentions.png', transparent=True)

In [ ]:
more_flow = np.maximum(flow_values*8 - res_adj_mat, 0)
moreflow_G = draw_attention_graph(more_flow,labels_to_index, n_layers=attentions_mat.shape[0], length=attentions_mat.shape[-1])
#plt.savefig('flow-raw_attentions.png', transparent=True)

In [ ]:
more_flow = np.maximum(res_adj_mat - flow_values*8, 0)
moreflow_G = draw_attention_graph(more_flow,labels_to_index, n_layers=attentions_mat.shape[0], length=attentions_mat.shape[-1])
#plt.savefig('flow-raw_attentions.png', transparent=True)

In [ ]:
att_sum_heads =  attentions_mat.sum(axis=1)/8
joint_attentions = compute_joint_attention(att_sum_heads, add_residual=True)
joint_att_adjmat, joint_labels_to_index = get_adjmat(mat=joint_attentions, input_tokens=['<cls>']+sentence.split())

G = draw_attention_graph(joint_att_adjmat,joint_labels_to_index, n_layers=joint_attentions.shape[0], length=joint_attentions.shape[-1])

In [ ]:
A = np.maximum(joint_att_adjmat*8 - res_adj_mat, 0)
draw_attention_graph(A,joint_labels_to_index, n_layers=joint_attentions.shape[0], length=joint_attentions.shape[-1])

In [ ]:
A = np.maximum(res_adj_mat - joint_att_adjmat*8, 0)
draw_attention_graph(A,joint_labels_to_index, n_layers=joint_attentions.shape[0], length=joint_attentions.shape[-1])

In [ ]:
length = att_mat.shape[-1]
final_layer_attention_raw = flow_values[6*length:,5*length:6*length]#joint_attentions[-1]#att_mat[-1]
relevance_attention_raw = final_layer_attention_raw[0]
rel_rank_att_raw = np.argsort(relevance_attention_raw)

print(rel_rank_att_raw, relevance_attention_raw[rel_rank_att_raw])
print("least important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[0]])
print("most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-1]])
print("2nd most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-2]])
print("3rd most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-3]])
print("4th most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-4]])
print("5th most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-5]])
print("6th most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-6]])

In [ ]:
final_layer_attention_raw = att_mat[-1]
relevance_attention_raw = final_layer_attention_raw[0]
rel_rank_att_raw = np.argsort(relevance_attention_raw)

print(rel_rank_att_raw, relevance_attention_raw[rel_rank_att_raw])
print("least important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[0]])
print("most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-1]])
print("2nd most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-2]])
print("3rd most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-3]])
print("4th most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-4]])

In [ ]:
final_layer_attention_raw = joint_attentions[-1]
relevance_attention_raw = final_layer_attention_raw[-1]
rel_rank_att_raw = np.argsort(relevance_attention_raw)

print(rel_rank_att_raw, relevance_attention_raw[rel_rank_att_raw])
print("least important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[0]])
print("most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-1]])
print("2nd most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-2]])
print("3rd most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-3]])
print("4th most important token:", (['<cls>']+sentence.split())[rel_rank_att_raw[-4]])

In [ ]:
index = 0
sentence =  task.databuilder.sentence_encoder().decode(x[index])#'<bos> the young girl who plays with the older kids is happy <eos>'
sentence = '<bos> according to contemporary accounts in the NNP NNPS the college ' + \
           'of cardinals was divided into NNS of charles NN ( VBZ NNP ) and the imperial party ( VBZ NNP ) , '+\
          'but the exact reconstruction of these parties <eos>'
sentence = '<bos> the young girl who plays with the older kids <eos>'
print(sentence)
#'<bos> the boys who play football with the old man <eos>'
encoded_sentence = task.databuilder.sentence_encoder().encode(sentence)

logits, extra = model_1.detailed_call(np.asarray([encoded_sentence]))
print((np.argmax(logits)))

In [19]:
sentences = []
all_atts = []
all_main_probs = []
all_index_probs = []
prob_fn = task.get_probs_fn()
count = 0
for x, y in iter(task.test_dataset):
    max_len = x.shape[1]
    main_logits, extra = model_1.detailed_call(x, training=False)
    outputs, last_state, presents, attentions, embeddings = extra
    _attentions = [att.numpy() for att in attentions]
    attentions = np.transpose(np.asarray(_attentions), (1,0,2,3,4))
    main_probs = prob_fn(main_logits, y, 1)
    batch_indexes = tf.range(len(y), dtype=tf.int64)
    indexes = tf.concat([batch_indexes[:,None], y[:,None]], axis=1)
    correct_main_probs = tf.gather_nd(main_probs, indexes).numpy()

    sentences.append(task.databuilder.sentence_encoder().decode(x[0]))
    all_atts.extend(attentions)
    all_main_probs.extend(correct_main_probs)
    all_index_probs.append([])
    print(count, max_len)
    # This loop can be optimized so that there is only one call...
    new_xz = []
    for i in np.arange(1,max_len-1):
        batch_size = tf.shape(x)[0]
        unktoken = task.databuilder.sentence_encoder().encode(constants.unk)
        unk = tf.reshape(tf.convert_to_tensor(unktoken, dtype=tf.int64)[None], (-1,1))
        unks = tf.tile(unk, (batch_size, 1))
        new_x = tf.concat([x[:,:i],unks, x[:,i+1:]], axis=-1)
        new_xz.extend(new_x)
    
    new_x = np.asarray(new_xz)
    logits = model_1(new_x, training=False)
    probs = prob_fn(logits, y, 1)
    
    batch_indexes = tf.range(len(probs), dtype=tf.int64)
    yz = tf.tile(y, (len(probs),))

    indexes = tf.concat([batch_indexes[:,None], yz[:,None]], axis=1)
    
    correct_probs = tf.gather_nd(probs, indexes).numpy()
    all_index_probs[-1].extend(abs(correct_main_probs - correct_probs))
    count += 1
    if count > 1000:
        break

0 6
1 6
2 5
3 12
4 13
5 6
6 8
7 8
8 4
9 4
10 4
11 7
12 21
13 6
14 7
15 5
16 15
17 4
18 7
19 4
20 5
21 6
22 8
23 11
24 5
25 5
26 18
27 5
28 20
29 17
30 10
31 7
32 11
33 9
34 4
35 5
36 8
37 4
38 32
39 6
40 5
41 5
42 21
43 5
44 6
45 8
46 5
47 22
48 9
49 8
50 11
51 11
52 5
53 4
54 5
55 3
56 10
57 11
58 25
59 8
60 19
61 4
62 25
63 13
64 3
65 4
66 13
67 5
68 6
69 8
70 13
71 4
72 6
73 4
74 20
75 31
76 8
77 9
78 21
79 9
80 6
81 14
82 27
83 6
84 14
85 11
86 4
87 5
88 4
89 6
90 20
91 10
92 15
93 14
94 5
95 6
96 4
97 19
98 4
99 18
100 13
101 32
102 5
103 5
104 3
105 6
106 5
107 4
108 20
109 10
110 3
111 6
112 24
113 10
114 9
115 4
116 15
117 7
118 6
119 12
120 5
121 4
122 5
123 7
124 28
125 3
126 5
127 14
128 4
129 22
130 31
131 10
132 8
133 4
134 21
135 4
136 4
137 16
138 4
139 30
140 15
141 16
142 6
143 9
144 10
145 4
146 4
147 43
148 10
149 26
150 4
151 4
152 6
153 4
154 6
155 7
156 4
157 24
158 4
159 9
160 14
161 8
162 9
163 4
164 7
165 6
166 7
167 4
168 8
169 11
170 3
171 5
172 7
173 5
174 1

In [21]:
def spearmanr(x, y):
    """ `x`, `y` --> pd.Series"""
    x = pd.Series(x)
    y = pd.Series(y)
    assert x.shape == y.shape
    rx = x.rank(method='dense')
    ry = y.rank(method='dense')
    d = rx - ry
    dsq = np.sum(np.square(d))
    n = x.shape[0]
    coef = 1. - (6. * dsq) / (n * (n**2 - 1.))
    return coef

def get_raw_att_relevance(full_att_mat, input_tokens, layer=-1):
    cls_index = 0
    return full_att_mat[layer].sum(axis=0)[cls_index]
    

def compute_node_flow(G, labels_to_index, input_nodes, output_nodes,length):
    number_of_nodes = len(labels_to_index)
    flow_values=np.zeros((number_of_nodes,number_of_nodes))
    for key in output_nodes:
        if key not in input_nodes:
            current_layer = int(labels_to_index[key] / length)
            pre_layer = current_layer - 1
            u = labels_to_index[key]
            for inp_node_key in input_nodes:
                v = labels_to_index[inp_node_key]
                flow_value = nx.maximum_flow_value(G,u,v)
                flow_values[u][pre_layer*length+v ] = flow_value
            flow_values[u] /= flow_values[u].sum()
            
    return flow_values

def get_flow_relevance(full_att_mat, input_tokens, layer):
    
    res_att_mat = full_att_mat.sum(axis=1)/8
    res_att_mat = res_att_mat + np.eye(res_att_mat.shape[1])[None,...]
    res_att_mat = res_att_mat / res_att_mat.sum(axis=-1)[...,None]

    res_adj_mat, res_labels_to_index = get_adjmat(mat=res_att_mat, input_tokens=input_tokens)
    
    A = res_adj_mat
    res_G=nx.from_numpy_matrix(A, create_using=nx.DiGraph())
    for i in np.arange(A.shape[0]):
        for j in np.arange(A.shape[1]):
            nx.set_edge_attributes(res_G, {(i,j): A[i,j]}, 'capacity')


    output_nodes = []
    input_nodes = []
    for key in res_labels_to_index:
        if 'L'+str(layer+1) in key:
            output_nodes.append(key)
        if res_labels_to_index[key] < full_att_mat.shape[-1]:
            input_nodes.append(key)
    
    flow_values = compute_node_flow(res_G, res_labels_to_index, input_nodes, output_nodes, length=full_att_mat.shape[-1])
    
    n_layers = full_att_mat.shape[0]
    length = full_att_mat.shape[-1]
    final_layer_attention_raw = flow_values[(layer+1)*length:,layer*length:(layer+1)*length]
    cls_index = 0
    relevance_attention_raw = final_layer_attention_raw[cls_index]

    return relevance_attention_raw
    
    
def get_joint_relevance(full_att_mat, input_tokens, layer):
    att_sum_heads =  full_att_mat.sum(axis=1)/8
    joint_attentions = compute_joint_attention(att_sum_heads, add_residual=True)
    relevance_attentions = joint_attentions[layer][0]
    return relevance_attentions

from tqdm import tqdm
for layer in np.arange(6):
    print("Layer: ", layer) 
#     r_corels, r_pvals = [], []
#     for i in np.arange(len(all_index_probs)):
#         aa = [0, 0]+all_index_probs[i]+[0]
#         bb = get_raw_att_relevance(all_atts[i], ['<cls>']+sentences[i].split(), layer=layer)
#         corel = spearmanr(aa,bb)
#         r_corels.append(corel) 

#     print("r:", np.mean(r_corels), np.std(r_corels))

#     j_corels, j_pvals = [], []
#     for i in np.arange(len(all_index_probs)):
#         aa = [0, 0]+all_index_probs[i]+[0]
#         bb = get_joint_relevance(all_atts[i], ['<cls>']+sentences[i].split(),layer=layer)
#         corel = spearmanr(aa,bb)
#         j_corels.append(corel) 
#     print("j:", np.mean(j_corels), np.std(j_corels))

    f_corels, f_pvals = [], []
    for i in tqdm(np.arange(len(all_index_probs))):
        aa = [0, 0]+all_index_probs[i]+[0]
        bb = get_flow_relevance(all_atts[i], ['<cls>']+sentences[i].split(),layer=layer)
        corel = spearmanr(aa,bb)
        f_corels.append(corel) 
    print("f:", np.mean(f_corels), np.std(f_corels))



  0%|          | 0/1001 [00:00<?, ?it/s]

Layer:  0


  0%|          | 1/1001 [00:00<03:04,  5.42it/s]

f: 0.1755475728467852 0.13963136005850138
Layer:  1


  0%|          | 0/1001 [00:00<?, ?it/s]

...

f: 0.5683504092257706 0.17421387258166213
Layer:  4


  0%|          | 1/1001 [00:00<02:20,  7.12it/s]

f: 0.562664291253102 0.1770877346129305
Layer:  5


100%|██████████| 1001/1001 [56:13<00:00,  3.37s/it] 

f: 0.5679205284029363 0.17561799076753754


In [10]:
np.save('all_probs', all_index_probs, allow_pickle=True)

NameError: name 'all_index_probs' is not defined

In [11]:
all_index_probs = np.load('all_probs.npy', allow_pickle=True)

In [18]:
import pickle
#pickle.dump(all_atts, open('all_atts', 'wb'))
pickle.load(open('all_atts', 'rb'))

EOFError: Ran out of input

In [ ]:


r_corels, r_pvals = [], []
for i in np.arange(len(all_index_probs)):
    aa = [0, 0]+all_index_probs[i]+[0]
    bb = get_raw_att_relevance(all_atts[i], ['<cls>']+sentences[i].split())
    corel = spearmanr(aa,bb)
    r_corels.append(corel) 

j_corels, j_pvals = [], []
for i in np.arange(len(all_index_probs)):
    aa = [0, 0]+all_index_probs[i]+[0]
    bb = get_joint_relevance(all_atts[i], ['<cls>']+sentences[i].split())
    corel = spearmanr(aa,bb)
    j_corels.append(corel) 

f_corels, f_pvals = [], []
for i in np.arange(len(all_index_probs)):
    aa = [0, 0]+all_index_probs[i]+[0]
    bb = get_flow_relevance(all_atts[i], ['<cls>']+sentences[i].split())
    corel = spearmanr(aa,bb)
    f_corels.append(corel) 

In [ ]:
f_corels, f_pvals = [], []
for i in np.arange(len(all_index_probs)):
    aa = [0, 0]+all_index_probs[i]+[0]
    bb = get_flow_relevance(all_atts[i], ['<cls>']+sentences[i].split())
    corel = spearmanr(aa,bb)
    f_corels.append(corel) 

In [ ]:
r_corels

In [ ]:
np.asarray(f_corels) > np.asarray(j_corels)

In [ ]:
np.asarray(f_corels) > np.asarray(r_corels)

In [ ]:
np.asarray(f_corels) > np.asarray(j_corels)

In [ ]:
f_corels

In [ ]:
r_corels, r_pvals = [], []
for i in np.arange(len(all_index_probs)):
    aa = [0, 0]+all_index_probs[i]+[0]
    bb = get_raw_att_relevance(all_atts[i], ['<cls>']+sentences[i].split())
    corel, p_value = spearmanr(aa,bb)
    r_corels.append(corel) 
    r_pvals.append(p_value)

In [ ]:
a = [1,2,3]
np.mean(a)
np.std(a)

In [ ]:


f_corels, f_pvals = [], []
for i in np.arange(len(all_index_probs)):
    aa = [0, 0]+all_index_probs[i]+[0]
    bb = get_flow_relevance(all_atts[i], ['<cls>']+sentences[i].split())
    corel = spearmanr(aa,bb)
    f_corels.append(corel) 
print("f:", np.mean(f_corels), np.std(f_corels))

In [ ]:
labels_to_index